# Setup

In [ ]:
from utils import *
from model import *
notebook_login()
%load_ext autoreload
%autoreload 2

# Load Datasets

In [2]:
opinions_datatset = load_dataset('raminass/opinions')
opinions_df = pd.DataFrame(opinions_datatset['train'])
opinions_df = opinions_df[opinions_df.category!='per_curiam']
del opinions_datatset

In [3]:
opinions_df.author_name.value_counts()

author_name
Justice Stevens      1181
Justice Rehnquist     752
Justice Brennan       732
Justice Scalia        718
Justice White         655
Justice Blackmun      654
Justice Marshall      611
Justice Thomas        551
Justice O'Connor      518
Justice Kennedy       461
Justice Powell        455
Justice Burger        417
Justice Breyer        416
Justice Ginsburg      407
Justice Stewart       328
Justice Souter        291
Justice Douglas       287
Justice Alito         239
Justice Sotomayor     175
Justice Kagan         110
Justice Roberts        94
Justice Black          59
Justice Harlan         50
Justice Gorsuch        47
Justice Kavanaugh      20
Name: count, dtype: int64

# Models

## All Periods

In [4]:
opinion_train, opinion_test = train_test_split(opinions_df, random_state=1984, stratify=opinions_df['author_name'])
id2label, label2id = get_labels_maping(opinions_df)
paragraphs_test = split_data(opinion_test, id2label, label2id )
paragraphs_train = split_data(opinion_train, id2label, label2id )

In [5]:
paragraphs_train_smpl = paragraphs_train.groupby('author_name',as_index = False,group_keys=False).apply(lambda s: s.sample(1500, replace=True))

In [8]:
ds_dict = {'train' : Dataset.from_pandas(paragraphs_train_smpl[["label", "text"]], preserve_index=False),
           'test' : Dataset.from_pandas(paragraphs_test[["label", "text"]], preserve_index=False)}
ds = DatasetDict(ds_dict)
tokenized_court = tokenize_dataset(ds)

Map:   0%|          | 0/37500 [00:00<?, ? examples/s]

Map:   0%|          | 0/17992 [00:00<?, ? examples/s]

In [ ]:
trainer = get_new_trainer(id2label, label2id, tokenized_court, epochs=20, model_name='1970-2020', batch_size=16)

In [ ]:
trainer.train()
trainer.push_to_hub()

In [48]:
# The Burger Court era, under the leadership of Warren E. Burger, lasted from June 23, 1969 to September 26, 1986; 13 justices served during this court.
burger_df = opinions_df[opinions_df.date_filed.between('1969-06-23', '1986-09-26')]
burger_df.author_name.value_counts()

author_name
Justice Brennan      575
per_curiam           539
Justice Marshall     463
Justice White        462
Justice Rehnquist    457
Justice Stevens      454
Justice Blackmun     439
Justice Powell       421
Justice Burger       417
Justice Stewart      328
Justice Douglas      287
Justice O'Connor     140
Justice Black         59
Justice Harlan        50
Name: count, dtype: int64

In [50]:
# The Rehnquist Court era, under the leadership of William Rehnquist, lasted from September 26, 1986 to September 3, 2005.
rehnquist_df = opinions_df[opinions_df.date_filed.between('1986-09-26', '2005-09-03')]
rehnquist_df.author_name.value_counts()

author_name
Justice Stevens      613
Justice Scalia       497
Justice O'Connor     376
Justice Kennedy      298
Justice Rehnquist    295
Justice Thomas       244
Justice Souter       239
Justice Blackmun     215
Justice White        192
Justice Ginsburg     192
per_curiam           188
Justice Breyer       185
Justice Brennan      157
Justice Marshall     148
Justice Powell        34
Name: count, dtype: int64

In [53]:
#  The Roberts Court era, under the leadership of John Roberts, began September 29, 2005, and is ongoing
roberts_df = opinions_df[opinions_df.date_filed > '2005-09-29']
roberts_df = roberts_df[~roberts_df.author_name.isin(["Justice O'Connor", 'Justice White'])]
roberts_df.author_name.value_counts()

author_name
Justice Thomas       307
Justice Alito        239
Justice Breyer       231
Justice Scalia       221
Justice Ginsburg     215
Justice Sotomayor    175
Justice Kennedy      163
Justice Stevens      114
Justice Kagan        110
Justice Roberts       94
Justice Souter        52
Justice Gorsuch       47
per_curiam            28
Justice Kavanaugh     20
Name: count, dtype: int64

In [79]:
# merge without adding new columns
op = pd.merge(paragraphs_df, opinion_test, how='inner', on=['author_name', 'case_name'])

(8007, 19)

In [80]:
op[op.author_name=='per_curiam'].shape

(117, 19)

In [30]:
paragraphs_test = split_data(opinion_test[opinion_test.category!='per_curiam'], id2label, label2id )
paragraphs_train = split_data(opinion_train[opinion_train.category!='per_curiam'], id2label, label2id )

In [ ]:
ds_dict = {'train' : Dataset.from_pandas(paragraphs_train[["label", "text"]], preserve_index=False),
           'test' : Dataset.from_pandas(paragraphs_test[["label", "text"]], preserve_index=False)}
ds = DatasetDict(ds_dict)

In [37]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Specify the BERT model and tokenizer
bert_model_name = "nlpaueb/legal-bert-small-uncased"
tokenizer = AutoTokenizer.from_pretrained(bert_model_name)

# Create model_1
model_1 = AutoModelForSequenceClassification.from_pretrained(bert_model_name, num_labels=2)

# Example input text for fine-tuning
input_text = "Fine-tuning example sentence."
labels = torch.tensor([1])  # Example labels (modify based on your task)

# Tokenize the input text
inputs = tokenizer(input_text, return_tensors="pt")

# Forward pass through model_1
output_1 = model_1(**inputs, labels=labels)

# Create model_2 with the same BERT transformer as model_1
model_2 = AutoModelForSequenceClassification.from_pretrained(bert_model_name, num_labels=3)

# Initialize model_2 with the pre-trained weights from model_1
model_2.bert = model_1.bert

# Forward pass through model_2 for fine-tuning
output_2 = model_2(**inputs, labels=labels)

# Optionally, you can further fine-tune model_2 on your specific task using the provided training data

# Display the logits for both models
print("Logits for Model 1:", output_1.logits)
print("Logits for Model 2:", output_2.logits)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nlpaueb/legal-bert-small-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nlpaueb/legal-bert-small-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Logits for Model 1: tensor([[0.2663, 0.3671]], grad_fn=<AddmmBackward0>)
Logits for Model 2: tensor([[ 0.2011, -0.3665, -0.2145]], grad_fn=<AddmmBackward0>)
